# Декораторы

# Задача 1: Суммы прогрессий

Необходимо реализовать функционал для подсчета суммы первых n + 1 - членов арифметической и геометрической прогрессии с возможностями настройки шага и значения первого члена. n соответствует числу вызовов функции по подсчету суммы. 

Предполагаемые сценарии использования:

```python

sum_arithmetic = make_arithmetic_progression_sum(first_member=2, step=0.5)
sum_geometric = make_geometric_progression_sum(first_member=1, step=0.5)

print(sum_arithmetic())
print(sum_arithmetic())
print('')
print(sum_geometric())
print(sum_geometric())
```

*Вывод*:
```console
4.5
7.5

1.5
1.75
```

In [1]:
def make_arithmetic_progression_sum(first_member: float, step: float):
    # ваш код
    pass


def make_geometric_progression_sum(first_member: float, step: float):
    # ваш код
    pass

# Задача 2: Среднее

Предположим, что мы занимаемся инвистициями и у нас есть некоторый портфель акций. Каждый день наш портфель приносит нам некоторый доход или убыток. Мы задались целью: каждый день фиксированного периода определять средний доход (или убыток), который мы получаем. С этой целью мы реализовали функцию get_avg(), принимающую на вход значение заработка на сегодняшний день. Наша функция вычисляет среднее в течении определнного фиксированного периода, скажем, 30 дней, после чего обнуляется и начинает вычислять среднее заново, но уже по новым значениям. 

Также у нас есть друзья инвесторы, которые оценили разработанный нами функционал и хотели бы заполучить свой экземпляр функции get_avg, для подсчета своего дохода в течении интересующего их промежутка времени.

Ваша задача: реализовать функционал, для получения произвольного числа независимых функций get_avg(). В момент создания функции сообщается длительность периода расчета среднего, по достижении которого среднее начинает расчитываться заново, а также наш начальный доход. При каждом вызове функции передается число - заработок в текущий день.

Предполагаемые сценарии использования:

```python

get_avg1 = make_averager(accumulation_period=2)
print(get_avg1(78))
print(get_avg1(-17))
print(get_avg1(52))
```

*Вывод*:
```console
78.0
30.5
52.0
```

In [1]:
# ваш код
def make_averager(accumulation_period: int):
    avg_sum = 0
    counter = 0
    def calculate(num: int):
        nonlocal avg_sum, counter
        if counter == accumulation_period:
            counter = 0
            avg_sum = 0
        avg_sum += num
        counter += 1
        return avg_sum/counter
    return calculate

In [6]:
get_avg1 = make_averager(accumulation_period=2)
print(get_avg1(78))
print(get_avg1(-17))
print(get_avg1(52))

78.0
30.5
52.0


# Задача 3: Сбор статистик

Предположим, что мы работаем в отделе аналитики некоторой компании. В компании также существуют другие отделы, которые разрабатывают некоторые функции для осуществления сложных вычислений. Также в нашей компании существует отдел планирования, который следит за исполнением сроков реализации той или иной функции, и в случае, если разработка затягивается, начинает торопить разработчиков. В таком случае разработчики пишут медленный код на скорую руку, что расстраивает заказчиков.

Наша задача, как аналитиков, собрать статистику по проблемным функциям. Нас интересует количество вызовов функции, а также среднее время выполнения функции. Все статистики собираются в отдельную базу данных - специальный единый словарь. Более того, статистика должна собираться не для всех функций, а только для функций, зарегестрированных в базе данных. Затем эта информация будет передана начальству, чтобы в скорейшее время заняться переписанием долгих и популярных функций.

Ваша задача реализовать функционал для регистрации функций в БД и сбора статистик. 

In [22]:
functions_register = {}

import time
from random import randint
from functools import wraps

def register(funk):
    if funk not in functions_register:
        functions_register[funk] = {
            'time_avg' : 0,
            'calls_count' : 0
        }
    return funk


def stats(funk):
    time_general = 0
    @wraps(funk)
    def count_time(*args, **kwargs):
        nonlocal time_general 
        t = time.time()
        result = funk(*args, **kwargs)
        time_general += time.time() - t
        if funk in functions_register:
            functions_register[funk]['calls_count'] += 1
            functions_register[funk]['time_avg'] = (
                time_general/functions_register[funk]['calls_count']
            )
        return result
    return count_time

@stats
@register
def do_something(*args):
    timer = randint(1, 10)
    time.sleep(timer)


In [23]:

print(do_something())
print(functions_register)
print(do_something())
print(functions_register)

None
{<function do_something at 0x000001FA0916FB00>: {'time_avg': 3.002067804336548, 'calls_count': 1}}
None
{<function do_something at 0x000001FA0916FB00>: {'time_avg': 5.002010345458984, 'calls_count': 2}}


# Задача 4: наивный LRU-кэш

Представим, что вы являетесь сотрудником департамента оптимизаций некоторой компании, занимающейся разработкой ПО для научных вычислений. Раздел аналитиков предоставил вам исследование, согласно которому значительная часть функций в вашей компании работает очень медленно и должна быть оптимизирована. Более того, согласно исследованию, вызовы этих функций в основном осуществляются с ограниченным множеством аргументов. Для оптимизации этих функций вы решили использовать LRU-кэш:

- вы заранее фиксируете размер кэша - памяти, выделенной для хранения результатов вычислений функции;  
- в кэше хранится следующая информация: аргументы вызова - результат;  
- помимо этого для каждой пары хранится время их последнего вызова;  
- в случае достижения объема кэша установленной границы удаляются значения, чьи времена последних вызовов являются самыми старыми;  

Для применения данного подхода ко всем проблемным функциям, не переписывая сами функции, вы решили реализовать параметрический декоратор, т.к. для разных функций требуется разный размер кеша.

Ваша задача: реализовать описанный декоратор.

In [50]:
import time

def lru_cash(cash_size: int):
    def apply_cash(func):
        args_info = {}
        time_info = {}

        def count_cashe(*args, **kwargs):
            nonlocal args_info, time_info
            key = args + tuple(kwargs.values())
            if key in args_info:
                time_info[key] = time.time()
            else:
                if cash_size == len(args_info):
                    least = min(time_info, key = lambda x: x[- 1])
                    del args_info[least]
                    del time_info[least]
                args_info[key] = func(*args, **kwargs)
                time_info[key] = time.time()

            print(args_info)
            print(time_info)

            return args_info[key]
        return count_cashe
    return apply_cash


@lru_cash(5)
def do_something(*args):
    timer = randint(0, 2)
    time.sleep(timer)
    return timer

In [51]:
do_something(1)
do_something(1, 2, 3)
do_something(1)
do_something(2)
do_something(3)
do_something(4)
do_something(5)

{(1,): 1}
{(1,): 1698422166.5465205}
{(1,): 1, (1, 2, 3): 0}
{(1,): 1698422166.5465205, (1, 2, 3): 1698422166.5465205}
{(1,): 1, (1, 2, 3): 0}
{(1,): 1698422166.5465205, (1, 2, 3): 1698422166.5465205}
{(1,): 1, (1, 2, 3): 0, (2,): 2}
{(1,): 1698422166.5465205, (1, 2, 3): 1698422166.5465205, (2,): 1698422168.5475986}
{(1,): 1, (1, 2, 3): 0, (2,): 2, (3,): 0}
{(1,): 1698422166.5465205, (1, 2, 3): 1698422166.5465205, (2,): 1698422168.5475986, (3,): 1698422168.5475986}
{(1,): 1, (1, 2, 3): 0, (2,): 2, (3,): 0, (4,): 2}
{(1,): 1698422166.5465205, (1, 2, 3): 1698422166.5465205, (2,): 1698422168.5475986, (3,): 1698422168.5475986, (4,): 1698422170.5488336}
{(1, 2, 3): 0, (2,): 2, (3,): 0, (4,): 2, (5,): 2}
{(1, 2, 3): 1698422166.5465205, (2,): 1698422168.5475986, (3,): 1698422168.5475986, (4,): 1698422170.5488336, (5,): 1698422172.5500824}


2